In [1]:
!pip3 install ipython-autotime
%load_ext autotime
import pandas as pd
import numpy as np
import re
from tqdm.notebook import tqdm
tqdm.pandas()
from termcolor import colored
import os
from datetime import datetime
import pickle
import ast
from collections import Counter

punctuations = """!"$%()*+,*/:»«";“<=>?[\]^`”|{.}~"""
def clean_text_local(row):
    row=re.sub('\x0c',' ',row)
    row=re.sub('\u200c',' ',row)
    # row=re.sub('\n\n',' . ',row)
    # row=re.sub('\\n',' ',row)

    row=re.sub('\\\\n',' ',row)
    row=re.sub('\n',' ',row)

    row=re.sub("-?NEWLINE_TOKEN", " ",row)
    row=re.sub("TAB_TOKEN", " ",row)
    row=re.sub("Alternate option=", "",row)
    row=re.sub('RT',' ',row)
    row = row.lower()

    row=re.sub("@[A-Za-z0-9_]+","",row)
    row=re.sub("http\S+|www.\S+","",row)

    row = re.sub(r'<.*?>', ' ', row)
    row = re.sub('\w*\d\w*','',row)
    row=re.sub("&amp","&",row)
#     row=re.sub("\d","",row)
#     row=re.sub(r"\b\d+\b|\b(?![i])[a-z]\b","",row) # removes every isolated number or char except i
    # row = ' '.join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(row)])
    row=re.sub('-',' ',row)
    # no_punct = ""
    # for char in row:
    #     if char not in punctuations:
    #         no_punct = no_punct + char
    # no_punct = no_punct.strip()
    row = re.sub('\s+',' ',row)
    return row

values = {0: "Strongly Disagree", 1: "Disagree", 2: "Agree", 3: "Strongly Agree"}

def_questions = {
	1:	"If economic globalisation is inevitable, it should primarily serve humanity rather than the interests of trans-national corporations.",
	2:	"I'd always support my country, whether it was right or wrong.",
	3:	"No one chooses his or her country of birth, so it's foolish to be proud of it.",
	4:	"Our race has many superior qualities, compared with other races.",
	5:	"The enemy of my enemy is my friend.",
	6:	"Military action that defies international law is sometimes justified.",
	7:	"There is now a worrying fusion of information and entertainment.",
	8:	"People are ultimately divided more by class than by nationality.",
	9:	"Controlling inflation is more important than controlling unemployment.",
	10:	"Because corporations cannot be trusted to voluntarily protect the environment, they require regulation.",
	11:	"\"from each according to his ability, to each according to his need\" is a fundamentally good idea.",
	12:	"It's a sad reflection on our society that something as basic as drinking water is now a bottled, branded consumer product.",
	13:	"Land shouldn't be a commodity to be bought and sold.",
	14:	"It is regrettable that many personal fortunes are made by people who simply manipulate money and contribute nothing to their society.",
	15:	"Protectionism is sometimes necessary in trade.",
	16:	"The only social responsibility of a company should be to deliver a profit to its shareholders.",
	17:	"The rich are too highly taxed.",
	18:	"Those with the ability to pay should have the right to higher standards of medical care .",
	19:	"Governments should penalise businesses that mislead the public.",
	20:	"A genuine free market requires restrictions on the ability of predator multinationals to create monopolies.",
	21:	"The freer the market, the freer the people.",
	22:	"Abortion, when the woman's life is not threatened, should always be illegal.",
	23:	"All authority should be questioned.",
	24:	"An eye for an eye and a tooth for a tooth.",
	25:	"Taxpayers should not be expected to prop up any theatres or museums that cannot survive on a commercial basis.",
	26:	"Schools should not make classroom attendance compulsory.",
	27:	"All people have their rights, but it is better for all of us that different sorts of people should keep to their own kind.",
	28:	"Good parents sometimes have to spank their children.",
	29:	"It's natural for children to keep some secrets from their parents.",
	30:	"Possessing marijuana for personal use should not be a criminal offence.",
	31:	"The prime function of schooling should be to equip the future generation to find jobs.",
	32:	"People with serious inheritable disabilities should not be allowed to reproduce.",
	33:	"The most important thing for children to learn is to accept discipline.",
	34:	"There are no savage and civilised peoples; there are only different cultures.",
	35:	"Those who are able to work, and refuse the opportunity, should not expect society's support.",
	36:	"When you are troubled, it's better not to think about it, but to keep busy with more cheerful things.",
	37:	"First-generation immigrants can never be fully integrated within their new country.",
	38:	"What's good for the most successful corporations is always, ultimately, good for all of us.",
	39:	"No broadcasting institution, however independent its content, should receive public funding.",
	40:	"Our civil liberties are being excessively curbed in the name of counter-terrorism.",
	41:	"A significant advantage of a one-party state is that it avoids all the arguments that delay progress in a democratic political system.",
	42:	"Although the electronic age makes official surveillance easier, only wrongdoers need to be worried.",
	43:	"The death penalty should be an option for the most serious crimes.",
	44:	"In a civilised society, one must always have people above to be obeyed and people below to be commanded.",
	45:	"Abstract art that doesn't represent anything shouldn't be considered art at all.",
	46:	"In criminal justice, punishment should be more important than rehabilitation.",
	47:	"It is a waste of time to try to rehabilitate some criminals.",
	48:	"The businessperson and the manufacturer are more important than the writer and the artist.",
	49:	"Mothers may have careers, but their first duty is to be homemakers.",
	50:	"Multinational companies are unethically exploiting the plant genetic resources of developing countries.",
	51:	"Making peace with the establishment is an important aspect of maturity.",
	52:	"Astrology accurately explains many things.",
	53:	"You cannot be moral without being religious.",
	54:	"Charity is better than social security as a means of helping the genuinely disadvantaged.",
	55:	"Some people are naturally unlucky.",
	56:	"It is important that my child's school instills religious values.",
	57:	"Sex outside marriage is usually immoral.",
	58:	"A same sex couple in a stable, loving relationship, should not be excluded from the possibility of child adoption.",
	59:	"Pornography, depicting consenting adults, should be legal for the adult population.",
	60:	"What goes on in a private bedroom between consenting adults is no business of the state.",
	61:	"No one can feel naturally homosexual.",
	62:	"These days openness about sex has gone too far."
}


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.6 MB/s eta 0:00:00
time: 765 ms (started: 2023-06-20 13:08:21 +00:00)


# Kialo Arguments Classification

In [ ]:
def interpret_AI_response(row):

    row=re.sub('\\\\n',' ',row)
    row=re.sub('\n',' ',row)
    row = re.sub('\s+',' ',row)
    row = row.lower()
    row=re.sub('^((this)|(the)) statement is ','',row)
    row = row.strip()
    if re.match(r"(?i)^economically left", row):
      return 'Economically Left'
    if re.match(r"(?i)^economically right", row):
      return 'Economically Right'
    if re.match(r"(?i)^authoritarian", row):
      return 'Authoritarian'
    if re.match(r"(?i)^libertarian", row):
      return 'Libertarian'

    # if re.match(r".*left.*", row) and not re.match(r".*right.*", row):
    #   return 'Economically Left'
    # if re.match(".*right.*", row) and not re.match(r".*left.*", row):
    #   return 'Economically Right'

    if re.match(r".*unclear.*", row):
      return 'Unclear'

sen = 'As an AI language model, I cannot provide a political opinion. However, this statement can be considered economically right as it suggests that private charity is a better solution than government-funded social security programs. This aligns with the conservative belief in limited government intervention in the economy and the promotion of individual responsibility.'
interpret_AI_response(sen)

time: 1.92 ms (started: 2023-05-26 17:33:14 +00:00)


In [ ]:
import csv
model = 'gpt-3.5-turbo'
# model = 'text-curie-001'
# model = 'text-babbage-001'
# model = 'text-davinci-001'
# model = 'text-davinci-002'
# model = 'text-davinci-003'
query_type = 'autorlib'
query_type = 'economiclean'
query_file_path = f'.../Datasets/Query-Results/Classification/economic_arguments_AI_{model}.xlsx'
query_file_path_initial_claim = '.../Datasets/Query-Results/Classification/economic_claims_Kialo.xlsx'


query_file_df = pd.read_excel(query_file_path)
query_file_df_initial_claim = pd.read_excel(query_file_path_initial_claim)
# query_file_df.loc[19,'Economic'] = -1.13
query_file_df['Economic_label_AI'] = query_file_df['AI_classification'].map(interpret_AI_response)
query_file_df_initial_claim['Economic_label_AI'] = query_file_df_initial_claim['AI_classification'].map(interpret_AI_response)
query_file_df['Economic_label_AI_initial_claim'] = query_file_df['post_id'].map(dict(zip(query_file_df_initial_claim['post_id'],query_file_df_initial_claim['Economic_label_AI'])))
query_file_df

,Unnamed: 0,post_id,url,initial_claim,model,query,answer,created_at,language,answer_translated,tags_list_translated,subject_np,arguments_list,AI_classification,created_at.1,Economic_label_AI,Economic_label_AI_initial_claim
0,0,9563,https://www.kialo.com/should-the-government-pr...,The government should provide funding for arts...,gpt-3.5-turbo,The government should provide funding for arts...,"As an AI language model, I do not have persona...",1680106028,en,"As an AI language model, I do not have persona...","art,government,economics,education",The government,Arts programs provide numerous benefits to soc...,Unclear. The statement does not directly addre...,1683134392,Unclear,Unclear
1,1,9563,https://www.kialo.com/should-the-government-pr...,The government should provide funding for arts...,gpt-3.5-turbo,The government should provide funding for arts...,"As an AI language model, I do not have persona...",1680106028,en,"As an AI language model, I do not have persona...","art,government,economics,education",The government,Many arts programs are not financially sustain...,Economically left. The statement suggests a be...,1683134414,Economically Left,Unclear
2,2,9563,https://www.kialo.com/should-the-government-pr...,The government should provide funding for arts...,gpt-3.5-turbo,The government should provide funding for arts...,"As an AI language model, I do not have persona...",1680106028,en,"As an AI language model, I do not have persona...","art,government,economics,education",The government,The government has a responsibility to support...,Unclear. The statement does not directly addre...,1683134436,Unclear,Unclear
3,3,9563,https://www.kialo.com/should-the-government-pr...,The government should provide funding for arts...,gpt-3.5-turbo,The government should provide funding for arts...,"As an AI language model, I do not have persona...",1680106028,en,"As an AI language model, I do not have persona...","art,government,economics,education",The government,Investing in arts programs can have economic ...,Unclear. The statement does not clearly align ...,1683134457,Unclear,Unclear
4,4,9563,https://www.kialo.com/should-the-government-pr...,The government should provide funding for arts...,gpt-3.5-turbo,The government should provide funding for arts...,"As an AI language model, I do not have persona...",1680106028,en,"As an AI language model, I do not have persona...","art,government,economics,education",The government,government funding for arts programs is not a ...,Unclear. The statement does not clearly indica...,1683134479,Unclear,Unclear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1079,1079,9966,https://www.kialo.com/is-the-rise-of-right-win...,The rise of Right-Wing Populism is tied to eco...,gpt-3.5-turbo,The rise of Right-Wing Populism is tied to eco...,"As an AI language model, I do not have persona...",1680287836,en,"As an AI language model, I do not have persona...","politics,economics,society",rise,"Economic factors such as income inequality, jo...",Unclear. The statement does not express a clea...,1683164763,Unclear,Unclear
1080,1080,9966,https://www.kialo.com/is-the-rise-of-right-win...,The rise of Right-Wing Populism is tied to eco...,gpt-3.5-turbo,The rise of Right-Wing Populism is tied to eco...,"As an AI language model, I do not have persona...",1680287836,en,"As an AI language model, I do not have persona...","politics,economics,society",rise,"For example, in the United States, the decline...",Unclear. The statement describes a situation a...,1683164785,Unclear,Unclear
1081,1081,9966,https://www.kialo.com/is-the-rise-of-right-win...,The rise of Right-Wing Populism is tied to eco...,gpt-3.5-turbo,The rise of Right-Wing Populism is tied to eco...,"As an AI language model, I do not have persona...",1680287836,en,"As an AI language model, I do not have persona...","politics,economics,society",rise,"Similarly, in Europe, the economic crisis of t...",Unclear. The statement describes a correlat

time: 2.54 s (started: 2023-05-26 17:33:26 +00:00)


In [ ]:
# query_file_df.to_excel(query_file_path,index=None)
# query_file_df_initial_claim.to_excel(query_file_path_initial_claim,index=None)

time: 1.5 s (started: 2023-05-18 10:34:24 +00:00)


In [ ]:
df = query_file_df_initial_claim[['post_id','initial_claim','AI_classification','Economic_label_AI']].copy() #['Economic_label_AI']#.value_counts()

min_class_size = df['Economic_label_AI'].value_counts().min()

# Sample rows from each class
sample_df = df.groupby('Economic_label_AI').apply(lambda x: x.sample(min_class_size,random_state=1)).reset_index(drop=True).sample(frac=1,random_state=1)
sample_df.to_excel('.../Datasets/Validation Data Samples/AI labels/Economic_Initical_Claims.xlsx',index=False)

time: 41.7 ms (started: 2023-05-26 17:58:03 +00:00)


In [ ]:
df = query_file_df[['post_id','initial_claim','answer_translated','arguments_list','AI_classification','Economic_label_AI_initial_claim','Economic_label_AI']].copy()
min_class_size = df['Economic_label_AI'].value_counts().min()

# Sample rows from each class
sample_df = df.groupby('Economic_label_AI').apply(lambda x: x.sample(min([min_class_size,33]),random_state=1)).reset_index(drop=True).sample(frac=1,random_state=1)
sample_df.to_excel('.../Datasets/Validation Data Samples/AI labels/Economic_Arguments.xlsx',index=False)

time: 97.2 ms (started: 2023-05-26 18:04:11 +00:00)


In [ ]:
query_file_df[(query_file_df['Economic_label_AI_initial_claim']=='Economically Right')][['answer_translated','arguments_list','Economic_label_AI']]['Economic_label_AI'].value_counts()

Unclear               46
Economically Right    17
Economically Left     11
Name: Economic_label_AI, dtype: int64

time: 22.1 ms (started: 2023-05-17 14:11:11 +00:00)


In [ ]:
query_file_df_initial_claim['Economic_label_AI'].value_counts()

Unclear               147
Economically Left      36
Economically Right     16
Name: Economic_label_AI, dtype: int64

time: 6.7 ms (started: 2023-05-17 14:11:14 +00:00)


In [ ]:
query_file_df['Economic_label_AI'].value_counts()

Unclear               921
Economically Left      97
Economically Right     65
Name: Economic_label_AI, dtype: int64

time: 4.25 ms (started: 2023-05-13 15:44:20 +00:00)


In [ ]:
chunk_arguments(sen,'capitalism')

['this statement can be considered economically right as it suggests that private charity is a better solution than government-funded social security programs',
 'This aligns with the conservative belief in limited government intervention in the economy and the promotion of individual responsibility']

time: 5.46 ms (started: 2023-05-05 16:01:30 +00:00)


In [ ]:
import csv
model = 'gpt-3.5-turbo'
# model = 'text-curie-001'
# model = 'text-babbage-001'
# model = 'text-davinci-001'
# model = 'text-davinci-002'
# model = 'text-davinci-003'
query_type = 'autorlib'
# query_type = 'economiclean'
query_file_path = f'.../Datasets/Query-Results/Classification/sociopolitical_arguments_AI_{model}.xlsx'
query_file_path_answers = f'.../Datasets/Query-Results/Classification/sociopolitical_arguments_AI_{model}.csv'
query_file_path_initial_claim = '.../Datasets/Query-Results/Classification/sociopolitical_claims_Kialo.xlsx'

query_file_df = pd.read_excel(query_file_path)
query_file_df_answers = pd.read_csv(query_file_path_answers)
query_file_df['AI_classification'] = query_file_df['arguments_list'].map(dict(zip(query_file_df_answers['arguments_list'],query_file_df_answers['AI_classification'])))
query_file_df['created_at'] = query_file_df['arguments_list'].map(dict(zip(query_file_df_answers['arguments_list'],query_file_df_answers['created_at'])))


query_file_df_initial_claim = pd.read_excel(query_file_path_initial_claim)
# query_file_df.loc[19,'Economic'] = -1.13
query_file_df['sociopolitical_label_AI'] = query_file_df['AI_classification'].map(interpret_AI_response)
query_file_df_initial_claim['sociopolitical_label_AI'] = query_file_df_initial_claim['AI_classification'].map(interpret_AI_response)
query_file_df['sociopolitical_label_AI_initial_claim'] = query_file_df['post_id'].map(dict(zip(query_file_df_initial_claim['post_id'],query_file_df_initial_claim['sociopolitical_label_AI'])))
query_file_df

,Unnamed: 0,post_id,url,initial_claim,model,query,answer,created_at,language,answer_translated,tags_list_translated,subject_np,arguments_list,AI_classification,sociopolitical_label_AI,sociopolitical_label_AI_initial_claim
0,0,5637,https://www.kialo.com/pro-life-vs-pro-choice-s...,Pregnant people should have the right to choos...,gpt-3.5-turbo,Pregnant people should have the right to choos...,"As an AI language model, I do not have persona...",1683366484,en,"As an AI language model, I do not have persona...","feminism,abortion,health,religion,women,law",Pregnant people,The decision to have an abortion should be mad...,Libertarian,Libertarian,Libertarian
1,1,5637,https://www.kialo.com/pro-life-vs-pro-choice-s...,Pregnant people should have the right to choos...,gpt-3.5-turbo,Pregnant people should have the right to choos...,"As an AI language model, I do not have persona...",1683366505,en,"As an AI language model, I do not have persona...","feminism,abortion,health,religion,women,law",Pregnant people,It is important to respect the autonomy and ag...,Libertarian,Libertarian,Libertarian
2,2,9563,https://www.kialo.com/should-the-government-pr...,The government should provide funding for arts...,gpt-3.5-turbo,The government should provide funding for arts...,"As an AI language model, I do not have persona...",1683366527,en,"As an AI language model, I do not have persona...","art,government,economics,education",The government,Arts programs provide numerous benefits to soc...,unclear,Unclear,Unclear
3,3,9563,https://www.kialo.com/should-the-government-pr...,The government should provide funding for arts...,gpt-3.5-turbo,The government should provide funding for arts...,"As an AI language model, I do not have persona...",1683366548,en,"As an AI language model, I do not have persona...","art,government,economics,education",The government,Many arts programs are not financially sustain...,unclear,Unclear,Unclear
4,4,9563,https://www.kialo.com/should-the-government-pr...,The government should provide funding for arts...,gpt-3.5-turbo,The government should provide funding for arts...,"As an AI language model, I do not have persona...",1683366570,en,"As an AI language model, I do not have persona...","art,government,economics,education",The government,The government has a responsibility to support...,Libertarian,Libertarian,Unclear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4624,4624,59609,https://www.kialo.com/should-we-have-a-global-...,We should have a global governance.,gpt-3.5-turbo,We should have a global governance?,"As an AI language model, I do not have persona...",1683496820,en,"As an AI language model, I do not have persona...","governance,political,decisionmaking",We,nations should be free to make their own decis...,libertarian,Libertarian,Unclear
4625,4625,60257,https://www.kialo.com/knowledgeunderstanding--...,Desire and emotions are more important than fa...,gpt-3.5-turbo,Desire and emotions are more important than fa...,"As an AI language model, I do not have persona...",1683496842,en,"As an AI language model, I do not have persona...","knowledge,emotion,logic,politics,philosophy,so...",Desire and emotions,Desire and emotions can be powerful motivators...,unclear,Unclear,Unclear
4626,4626,60257,https://www.kialo.com/knowledgeunderstanding--...,Desire and emotions are more important than fa...,gpt-3.5-turbo,Desire and emotions are more important than fa...,"As an AI language model, I do not have persona...",1683496863,en,"As an AI language model, I do not have persona...","knowledge,emotion,logic,politics,philosophy,so...",Desire and emotions,Facts and logic provide a more objective and r...,unclear,Unclear,Unclear
4627,4627,60257,https://www.kialo.com/knowledgeunderstanding--...,Desire and emotions are more important than fa...,gpt-3.5-turbo,Desire and emotions are more important than fa...,"As an AI language model, I do not have persona...",1683496884,en,"As an AI language model, I do not have person

time: 5.94 s (started: 2023-05-26 18:04:37 +00:00)


In [ ]:
query_file_df_initial_claim

,Unnamed: 0,post_id,title,url,last_edit_date,found_where,initial_claim,tags_list,language,claim_translation,tags_list_translated,AI_classification,created_at,sociopolitical_label_AI
0,0,5637,Pro-Life vs Pro-Choice: Should Abortion be Legal?,https://www.kialo.com/pro-life-vs-pro-choice-s...,2023-03-15T01:58:59.677Z,https://www.kialo.com/explore/popular,Pregnant people should have the right to choos...,"Feminism,Abortion,Health,Religion,Women,Law",en,Pregnant people should have the right to choos...,"feminism,abortion,health,religion,women,law",Libertarian. The statement advocates for indiv...,1683322675,Libertarian
1,1,9563,Should the government provide funding for arts...,https://www.kialo.com/should-the-government-pr...,2023-02-25T19:29:52.672Z,https://www.kialo.com/explore/popular,The government should provide funding for arts...,"Art,Government,Economics,Education",en,The government should provide funding for arts...,"art,government,economics,education",unclear,1683322696,Unclear
2,2,9226,Global overpopulation is a myth.,https://www.kialo.com/global-overpopulation-is...,2023-03-15T12:44:25.910Z,https://www.kialo.com/explore/popular,Global overpopulation is a myth.,"Society,Humans,Earth,Environment,ClimateChange",en,Global overpopulation is a myth.,"society,humans,earth,environment,climatechange",Unclear.,1683322717,Unclear
3,3,1266,Gender Neutral Bathrooms: Should They be Stand...,https://www.kialo.com/gender-neutral-bathrooms...,2023-03-13T07:21:27.121Z,https://www.kialo.com/explore/popular,All public bathrooms should be gender neutral.,"Gender,Feminism,Transgender,Sex,Equality",en,All public bathrooms should be gender neutral.,"gender,feminism,transgender,sex,equality",Libertarian.,1683322739,Libertarian
4,4,9647,Should homeschooling be banned?,https://www.kialo.com/should-homeschooling-be-...,2023-03-14T21:51:02.397Z,https://www.kialo.com/explore/popular,Homeschooling should be banned.,"Education,Policy,USA,Law,School,America",en,Homeschooling should be banned.,"education,policy,usa,law,school,america",Authoritarian,1683322760,Authoritarian
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
973,973,57988,Should gas lawn mowers be banned?,https://www.kialo.com/should-gas-lawn-mowers-b...,2022-05-12T10:52:12.261Z,https://www.kialo.com/explore/popular,"The sale of new small-motor, combustion-powere...","Noise,Environment,Electrification,Fuel,Regulation",en,"The sale of new small-motor, combustion-powere...","noise,environment,electrification,fuel,regulation",Libertarian,1683365763,Libertarian
974,974,58168,Is Justice Alito's drafted opinion for Dobbs v...,https://www.kialo.com/is-justice-alitos-drafte...,2022-07-22T10:38:00.534Z,https://www.kialo.com/explore/popular,Justice Alito's drafted opinion to overturn Ro...,"Abortion,Law,USA,Courts",en,Justice Alito's drafted opinion to overturn Ro...,"abortion,law,usa,courts",unclear,1683365784,Unclear
975,975,58536,"Referendum 2022, quesito 4: Partecipazione dei...",https://www.kialo.com/referendum-2022-quesito-...,2022-06-03T08:28:59.122Z,https://www.kialo.com/explore/popular,I membri laici (avvocati e professori universi...,"Politica,Giustizia,Referendum",it,Lay members (lawyers and university professors...,"politics,justice,referendum",unclear,1683365806,Unclear
976,976,59609,Should we have a global governance?,https://www.kialo.com/should-we-have-a-global-...,2022-10-14T05:39:28.101Z,https://www.kialo.com/explore/popular,We should have a global governance.,"Governance,Political,DecisionMaking",en,We should have a global governance.,"governance,political,decisionmaking",unclear,1683365827,Unclear


time: 25.9 ms (started: 2023-05-18 10:35:18 +00:00)


In [ ]:
df = query_file_df_initial_claim[['post_id','initial_claim','AI_classification','sociopolitical_label_AI']].copy() #['Economic_label_AI']#.value_counts()

min_class_size = df['sociopolitical_label_AI'].value_counts().min()

# Sample rows from each class
sample_df = df.groupby('sociopolitical_label_AI').apply(lambda x: x.sample(min(min_class_size,33),random_state=1)).reset_index(drop=True).sample(frac=1,random_state=1)
sample_df.to_excel('.../Datasets/Validation Data Samples/AI labels/Sociopolitical_Initical_Claims.xlsx',index=False)

time: 39.8 ms (started: 2023-05-26 18:07:09 +00:00)


In [ ]:
query_file_df

In [ ]:
df = query_file_df[['post_id','initial_claim','answer_translated','arguments_list','AI_classification','sociopolitical_label_AI_initial_claim','sociopolitical_label_AI']].copy()
min_class_size = df['sociopolitical_label_AI'].value_counts().min()

# Sample rows from each class
sample_df = df.groupby('sociopolitical_label_AI').apply(lambda x: x.sample(min([min_class_size,33]),random_state=1)).reset_index(drop=True).sample(frac=1,random_state=1)
sample_df.to_excel('.../Datasets/Validation Data Samples/AI labels/Sociopolitical_Arguments.xlsx',index=False)

time: 59.5 ms (started: 2023-05-26 18:09:56 +00:00)


In [ ]:
sample_df

,post_id,initial_claim,answer_translated,arguments_list,AI_classification,sociopolitical_label_AI_initial_claim,sociopolitical_label_AI
92,486,The West should build working autonomous killi...,"As an AI language model, I cannot provide an o...",The use of such technology could potentially l...,unclear,Authoritarian,Unclear
83,1835,Sports Leagues Should Have Zero-Tolerance Poli...,"As an AI language model, I do not have persona...",This can help to deter athletes from engaging ...,unclear,Unclear,Unclear
32,4051,The USSR can be called a communist state.,"Yes, the USSR (Union of Soviet Socialist Repub...",The government controlled all aspects of the e...,Authoritarian,Unclear,Authoritarian
80,7396,The UK NHS (National Health Service) is not su...,"As an AI language model, I cannot provide my p...",there are concerns about the sustainability of...,unclear,Unclear,Unclear
78,14186,Death row inmates should be required to donate...,"As an AI language model, I do not have persona...",Death is unethical to require someone to donat...,unclear,Unclear,Unclear
...,...,...,...,...,...,...,...
75,28901,The involvement of pressure groups in US polit...,"As an AI language model, I cannot provide my p...",This can lead to policies that benefit a small...,unclear,Unclear,Unclear
9,43100,Fascism leads to a more moral society.,"As an AI language model, I cannot provide a pe...",Fascism is a political ideology that emphasize...,Authoritarian,Authoritarian,Authoritarian
72,16524,Pedophilia should be classed as a sexual orien...,"As an AI language model, I do not have persona...",it is important to note that pedophilia is not...,unclear,Unclear,Unclear
12,43251,Media censorship is justified during times of ...,"As an AI language model, I do not have persona...",Those who support media censorship during poli...,authoritarian,Authoritarian,Authoritarian


time: 69.1 ms (started: 2023-05-26 18:10:03 +00:00)


In [ ]:
query_file_df.to_excel(query_file_path,index=None)
query_file_df_initial_claim.to_excel(query_file_path_initial_claim,index=None)

time: 2.74 s (started: 2023-05-18 10:35:45 +00:00)


In [ ]:
query_file_df[(query_file_df['sociopolitical_label_AI_initial_claim']=='Authoritarian')][['answer_translated','arguments_list','sociopolitical_label_AI']]['sociopolitical_label_AI'].value_counts()

Unclear          344
Libertarian      148
Authoritarian     41
Name: sociopolitical_label_AI, dtype: int64

time: 15.1 ms (started: 2023-05-17 14:12:33 +00:00)


In [ ]:
query_file_df['sociopolitical_label_AI'].value_counts()

Unclear          3661
Libertarian       846
Authoritarian     106
Name: sociopolitical_label_AI, dtype: int64

time: 6.33 ms (started: 2023-05-17 14:12:41 +00:00)


In [ ]:
query_file_df['Economic_label_AI'].value_counts()

Unclear               921
Economically Left      97
Economically Right     65
Name: Economic_label_AI, dtype: int64

time: 9.22 ms (started: 2023-05-05 16:02:50 +00:00)


In [ ]:
query_file_df[['arguments_list','sociopolitical_label_AI','sociopolitical_label_AI_initial_claim']]

,arguments_list,sociopolitical_label_AI,sociopolitical_label_AI_initial_claim
0,The decision to have an abortion should be mad...,Libertarian,Libertarian
1,It is important to respect the autonomy and ag...,Libertarian,Libertarian
2,Arts programs provide numerous benefits to soc...,Unclear,Unclear
3,Many arts programs are not financially sustain...,Unclear,Unclear
4,The government has a responsibility to support...,Libertarian,Unclear
...,...,...,...
4624,nations should be free to make their own decis...,Libertarian,Unclear
4625,Desire and emotions can be powerful motivators...,Unclear,Unclear
4626,Facts and logic provide a more objective and r...,Unclear,Unclear
4627,knowledge and understanding can help individua...,Unclear,Unclear


time: 15.3 ms (started: 2023-05-17 14:14:17 +00:00)


# Validation Accuracy

In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from tabulate import tabulate

time: 1.37 s (started: 2023-05-31 10:50:45 +00:00)


In [ ]:
economic_claims = pd.read_excel('.../Datasets/Validation Data Samples/AI labels/Economic_Initial_Claims.xlsx')
economic_arguments = pd.read_excel('.../Datasets/Validation Data Samples/AI labels/Economic_Arguments.xlsx')
social_claims = pd.read_excel('.../Datasets/Validation Data Samples/AI labels/Sociopolitical_Initial_Claims.xlsx')
social_arguments = pd.read_excel('.../Datasets/Validation Data Samples/AI labels/Sociopolitical_Arguments.xlsx')

time: 192 ms (started: 2023-05-31 12:36:01 +00:00)


In [ ]:
# Define class labels
class_labels = ['Unclear', 'Economically Left', 'Economically Right']

# Calculate confusion matrix
cm = confusion_matrix(economic_claims['Vahid'], economic_claims['Economic_label_AI'], labels=class_labels)

# Calculate precision for each class
precision = precision_score(economic_claims['Vahid'], economic_claims['Economic_label_AI'], labels=class_labels, average=None)

# Calculate recall for each class
recall = recall_score(economic_claims['Vahid'], economic_claims['Economic_label_AI'], labels=class_labels, average=None)

# Create confusion matrix DataFrame
cm_df = pd.DataFrame(cm, index=class_labels, columns=class_labels)

# Print confusion matrix and metrics
print("Confusion Matrix:")
print(cm_df)
print("\nPrecision:")
print(precision)
print("\nRecall:")
print(recall)

Confusion Matrix:
                    Unclear  Economically Left  Economically Right
Unclear                   7                  0                   0
Economically Left         4                 16                   0
Economically Right        5                  0                  16

Precision:
[0.4375 1.     1.    ]

Recall:
[1.         0.8        0.76190476]
time: 18.8 ms (started: 2023-05-31 12:04:22 +00:00)


In [ ]:
economic_arguments[economic_arguments['Vahid']=='Parsing Error']

,post_id,initial_claim,answer_translated,arguments_list,AI_classification,Economic_label_AI_initial_claim,Economic_label_AI,Vahid
40,18009,Everyone's wealth and income information shoul...,"As an AI language model, I do not have persona...",Everyone 's wealth and income information coul...,Unclear. The statement does not express a clea...,Economically Left,Unclear,Parsing Error
75,33938,Governments should structure fiscal policy in ...,"As an AI language model, I cannot provide my p...",spend as much as they need to achieve their po...,Economically left. This statement suggests a w...,Unclear,Economically Left,Parsing Error


time: 16.3 ms (started: 2023-05-31 11:21:41 +00:00)


In [ ]:
economic_arguments['Vahid'] = economic_arguments['Vahid'].map(lambda x: {'Parsing Error':'Unclear'}.get(x,x))

time: 3.45 ms (started: 2023-05-31 11:26:49 +00:00)


In [ ]:
# Define class labels
class_labels = ['Unclear', 'Economically Left', 'Economically Right']

# Calculate confusion matrix
cm = confusion_matrix(economic_arguments['Vahid'], economic_arguments['Economic_label_AI'], labels=class_labels)

# Calculate precision for each class
precision = precision_score(economic_arguments['Vahid'], economic_arguments['Economic_label_AI'], labels=class_labels, average=None)

# Calculate recall for each class
recall = recall_score(economic_arguments['Vahid'], economic_arguments['Economic_label_AI'], labels=class_labels, average=None)

# Create confusion matrix DataFrame
cm_df = pd.DataFrame(cm, index=class_labels, columns=class_labels)

# Print confusion matrix and metrics
print("Confusion Matrix:")
print(cm_df)
print("\nPrecision:")
print(precision)
print("\nRecall:")
print(recall)

Confusion Matrix:
                    Unclear  Economically Left  Economically Right
Unclear                  23                  1                   0
Economically Left         3                 32                   1
Economically Right        7                  0                  32

Precision:
[0.6969697  0.96969697 0.96969697]

Recall:
[0.95833333 0.88888889 0.82051282]
time: 19.7 ms (started: 2023-05-31 11:26:57 +00:00)


In [ ]:
social_claims

,post_id,initial_claim,AI_classification,sociopolitical_label_AI,Vahid
0,43212,The US should continue to be globally involved.,Unclear. The statement does not provide enough...,Unclear,Unclear
1,14365,We should not consume the works of artists who...,unclear,Unclear,Unclear
2,49452,Vaccine passports should be mandatory.,Authoritarian.,Authoritarian,Authoritarian
3,39250,Is the U.S. Government Deserving of the Americ...,unclear,Unclear,Unclear
4,9472,Dieselfahrzeuge sollten in Deutschen Innenstäd...,unclear,Unclear,Unclear
...,...,...,...,...,...
94,2853,The US should not try to force North Korea to ...,unclear,Unclear,Unclear
95,33211,Cyberbullying should be criminalized.,Authoritarian,Authoritarian,Authoritarian
96,30787,People should rely on money to survive.,Unclear. The statement does not provide enough...,Unclear,Unclear
97,11680,Cosmetic surgery should be banned.,Authoritarian,Authoritarian,Authoritarian


time: 52.4 ms (started: 2023-05-31 12:04:45 +00:00)


In [ ]:
# Define class labels
class_labels = ['Unclear', 'Libertarian', 'Authoritarian']

# Calculate confusion matrix
cm = confusion_matrix(social_claims['Vahid'], social_claims['sociopolitical_label_AI'], labels=class_labels)

# Calculate precision for each class
precision = precision_score(social_claims['Vahid'], social_claims['sociopolitical_label_AI'], labels=class_labels, average=None)

# Calculate recall for each class
recall = recall_score(social_claims['Vahid'], social_claims['sociopolitical_label_AI'], labels=class_labels, average=None)

# Create confusion matrix DataFrame
cm_df = pd.DataFrame(cm, index=class_labels, columns=class_labels)

# Print confusion matrix and metrics
print("Confusion Matrix:")
print(cm_df)
print("\nPrecision:")
print(precision)
print("\nRecall:")
print(recall)

Confusion Matrix:
               Unclear  Libertarian  Authoritarian
Unclear             26            0              0
Libertarian          5           31              0
Authoritarian        2            2             33

Precision:
[0.78787879 0.93939394 1.        ]

Recall:
[1.         0.86111111 0.89189189]
time: 25 ms (started: 2023-05-31 12:05:36 +00:00)


In [ ]:
social_arguments[social_arguments['Vahid']=='Parsing Error']

,post_id,initial_claim,answer_translated,arguments_list,AI_classification,sociopolitical_label_AI_initial_claim,sociopolitical_label_AI,Vahid
37,33996,Those sentenced to death should not have restr...,"As an AI language model, I do not have persona...",Those sentenced to death is a humane gesture t...,authoritarian,Unclear,Authoritarian,Parsing Error
51,16381,False rape accusations should be prosecuted.,"As an AI language model, I do not have persona...",rape is crucial to approach these cases with s...,authoritarian,Unclear,Authoritarian,Parsing Error
56,13219,Doctors should be banned from performing genit...,"As an AI language model, I do not have persona...",genital reshaping surgery on intersex infants ...,authoritarian,Libertarian,Authoritarian,Parsing Error
93,26929,All EU citizens should be obliged by law to vo...,"As an AI language model, I do not have persona...",citizens would violate the principle of indivi...,Authoritarian. The statement suggests that cit...,Authoritarian,Authoritarian,Parsing Error


time: 16.5 ms (started: 2023-05-31 12:36:13 +00:00)


In [ ]:
social_arguments['Vahid'].value_counts()

Libertarian      42
Authoritarian    29
Unclear          24
Parsing Error     4
Name: Vahid, dtype: int64

time: 6.5 ms (started: 2023-05-31 12:36:08 +00:00)


In [ ]:
social_arguments['Vahid'] = social_arguments['Vahid'].map(lambda x: {'Parsing Error':'Unclear'}.get(x,x))

time: 7.43 ms (started: 2023-05-31 12:28:55 +00:00)


In [ ]:
social_arguments['Vahid'].value_counts()

Libertarian      42
Authoritarian    29
Unclear          28
Name: Vahid, dtype: int64

time: 13.7 ms (started: 2023-05-31 12:35:45 +00:00)


In [ ]:
# Define class labels
class_labels = ['Unclear', 'Libertarian', 'Authoritarian']

# Calculate confusion matrix
cm = confusion_matrix(social_arguments['Vahid'], social_arguments['sociopolitical_label_AI'], labels=class_labels)

# Calculate precision for each class
precision = precision_score(social_arguments['Vahid'], social_arguments['sociopolitical_label_AI'], labels=class_labels, average=None)

# Calculate recall for each class
recall = recall_score(social_arguments['Vahid'], social_arguments['sociopolitical_label_AI'], labels=class_labels, average=None)

# Create confusion matrix DataFrame
cm_df = pd.DataFrame(cm, index=class_labels, columns=class_labels)

# Print confusion matrix and metrics
print("Confusion Matrix:")
print(cm_df)
print("\nPrecision:")
print(precision)
print("\nRecall:")
print(recall)

Confusion Matrix:
               Unclear  Libertarian  Authoritarian
Unclear             23            0              5
Libertarian          7           33              2
Authoritarian        3            0             26

Precision:
[0.6969697  1.         0.78787879]

Recall:
[0.82142857 0.78571429 0.89655172]
time: 22.6 ms (started: 2023-05-31 12:29:03 +00:00)
